In [20]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus


In [21]:
# Credenciales base 2
user = "postgres"
password = "admin123"   # tu contraseña real
host = "localhost"
port = "5432"
database = "ecommerce"

In [22]:
# Codificar la contraseña para URL
password_encoded = quote_plus(password)

# Crear motor de conexión
engine = create_engine(
    f"postgresql+psycopg2://{user}:{password_encoded}@{host}:{port}/{database}"
)

# Leer CSV
df = pd.read_csv("olist_customers_dataset.csv")

# Subir a PostgreSQL
df.to_sql("counterfeit_transactions", engine, if_exists="replace", index=False)

print("✅ CSV cargado correctamente en PostgreSQL")

✅ CSV cargado correctamente en PostgreSQL


# ## Crear tabla en PostgreSQL y cargar datos
# Usamos `pandas.to_sql` para cargar todo el dataset a la base `olist` en la tabla `olist_customers`.

In [23]:
df.to_sql("olist_customers", engine, if_exists="replace", index=False)
print("✅ Datos cargados en la tabla 'olist_customers'")

✅ Datos cargados en la tabla 'olist_customers'


# ## DataFrame 1 – Clientes (`df_clientes`)
# Contiene los identificadores principales de clientes.
# - customer_id
# - customer_unique_id
# - customer_zip_code_prefix
#
# **Filtros:**
# 1. Clientes únicos por `customer_unique_id`.
# 2. Clientes con ZIP > 90000.
# 3. Clientes duplicados en `customer_id`.


In [24]:
df_clientes = df[["customer_id","customer_unique_id","customer_zip_code_prefix"]].copy()
print("Clientes:", df_clientes.shape)
df_clientes.head()

# Filtros
clientes_unicos = df_clientes.drop_duplicates(subset=["customer_unique_id"])
clientes_zip_altos = df_clientes[df_clientes["customer_zip_code_prefix"] > 90000]
clientes_duplicados = df_clientes[df_clientes.duplicated(subset=["customer_id"], keep=False)]

Clientes: (99441, 3)


# ## DataFrame 2 – Ubicaciones (`df_ubicaciones`)
# Información de localización.
# - customer_city
# - customer_state
#
# **Filtros:**
# 1. Clientes del estado "SP".
# 2. Clientes de ciudades con más de 500 registros.
# 3. Clientes de estados diferentes de "SP" y "RJ".

In [25]:
df_ubicaciones = df[["customer_id","customer_city","customer_state"]].copy()
print("Ubicaciones:", df_ubicaciones.shape)
df_ubicaciones.head()

# Filtros
clientes_sp = df_ubicaciones[df_ubicaciones["customer_state"] == "SP"]
clientes_ciudades_top = df_ubicaciones[df_ubicaciones["customer_city"].isin(
    df_ubicaciones["customer_city"].value_counts().head(10).index
)]
clientes_otro_estado = df_ubicaciones[~df_ubicaciones["customer_state"].isin(["SP","RJ"])]


Ubicaciones: (99441, 3)


# ## DataFrame 3 – Identificadores (`df_identificadores`)
# Validación de consistencia entre IDs y duplicados.
# - customer_id
# - customer_unique_id
#
# **Filtros:**
# 1. IDs únicos → conteo de `customer_id`.
# 2. Revisión de duplicados en `customer_unique_id`.
# 3. Top 10 IDs con más repeticiones.

In [26]:
df_identificadores = df[["customer_id","customer_unique_id"]].copy()

# Filtros
total_ids_unicos = df_identificadores["customer_id"].nunique()
duplicados_unique_id = df_identificadores[df_identificadores.duplicated(subset=["customer_unique_id"], keep=False)]
top_ids = df_identificadores["customer_unique_id"].value_counts().head(10)

print("Total customer_id únicos:", total_ids_unicos)
print("Duplicados en customer_unique_id:", duplicados_unique_id.shape)
print("Top 10 customer_unique_id repetidos:")
print(top_ids)

Total customer_id únicos: 99441
Duplicados en customer_unique_id: (6342, 2)
Top 10 customer_unique_id repetidos:
customer_unique_id
8d50f5eadf50201ccdcedfb9e2ac8455    17
3e43e6105506432c953e165fb2acf44c     9
6469f99c1f9dfae7733b25662e7f1782     7
ca77025e7201e3b30c44b472ff346268     7
1b6c7548a2a1f9037c1fd3ddfed95f33     7
12f5d6e1cbf93dafd9dcc19095df0b3d     6
dc813062e0fc23409cd255f7f53c7074     6
47c1a3033b8b77b3ab6e109eb4d5fdf3     6
de34b16117594161a6a89c50b289d35a     6
63cfc61cee11cbe306bff5857d00bfe4     6
Name: count, dtype: int64
